In [2]:
%pip install scikit-learn

  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl (44.5 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import mysql.connector

In [3]:
# Conexión a la base de datos
conexion = mysql.connector.connect(
    host="127.0.0.1",
    user="root",  
    password="Mysql123",
    database="yfinance"  
)

# Carga de datos desde una consulta SQL a un DataFrame
query = "SELECT precios_historicos.precio_apertura,precios_historicos.precio_cierre,precios_historicos.maximo,precios_historicos.minimo,precios_historicos.volumen,empresas_sp500.sector,empresas_sp500.industria FROM precios_historicos INNER JOIN empresas_sp500 ON precios_historicos.id_empresa = empresas_sp500.id_empresa;"
df = pd.read_sql(query, conexion)





C:\Users\Rober\AppData\Local\Temp\ipykernel_25184\462899246.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexion)


In [4]:
conexion = mysql.connector.connect(
    host="127.0.0.1",
    user="root",  
    password="Mysql123",
    database="yfinance")

query = "SELECT precios_historicos.precio_apertura,precios_historicos.precio_cierre,precios_historicos.maximo,precios_historicos.minimo,precios_historicos.volumen,empresas_sp500.sector,empresas_sp500.industria FROM precios_historicos INNER JOIN empresas_sp500 ON precios_historicos.id_empresa = empresas_sp500.id_empresa;"
empresas_df = pd.read_sql(query, conexion)

### encoder sector

label_encoder_sector = LabelEncoder()
empresas_df['sector_encoded'] = label_encoder_sector.fit_transform(empresas_df['sector'])

### encoder industria

label_encoder_industria = LabelEncoder()
empresas_df['industria_encoded'] = label_encoder_industria.fit_transform(empresas_df['industria'])

###

empresas_df.head(5)

C:\Users\Rober\AppData\Local\Temp\ipykernel_25184\3439496553.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empresas_df = pd.read_sql(query, conexion)


,precio_apertura,precio_cierre,maximo,minimo,volumen,sector,industria,sector_encoded,industria_encoded
0,20.1681,19.8138,20.2600,19.7482,2599386,Industrials,Industrial Conglomerates,6,67
1,19.4989,19.0265,19.9057,19.0265,3245705,Industrials,Industrial Conglomerates,6,67
2,19.1315,19.5776,20.2075,19.1315,4424482,Industrials,Industrial Conglomerates,6,67
3,19.8007,21.1523,21.5197,19.8007,7147057,Industrials,Industrial Conglomerates,6,67
4,21.2310,21.5721,21.7952,20.9817,4905035,Industrials,Industrial Conglomerates,6,67


In [5]:
# Simular datos de ejemplo
np.random.seed(42)

x = empresas_df[['precio_apertura','precio_cierre','maximo','minimo','volumen']]
y = empresas_df[['sector_encoded','industria_encoded']]

# Ejecutar K-Means para agrupar las acciones
kmeans = KMeans(n_clusters=5, random_state=42)
empresas_df = kmeans.fit_predict(empresas_df[['precio_apertura','precio_cierre','maximo','minimo','volumen']])

print(df.head())  # Visualizar las características y el cluster asignado

conexion.close()

   precio_apertura  precio_cierre   maximo   minimo  volumen       sector  \
0          20.1681        19.8138  20.2600  19.7482  2599386  Industrials   
1          19.4989        19.0265  19.9057  19.0265  3245705  Industrials   
2          19.1315        19.5776  20.2075  19.1315  4424482  Industrials   
3          19.8007        21.1523  21.5197  19.8007  7147057  Industrials   
4          21.2310        21.5721  21.7952  20.9817  4905035  Industrials   

                  industria  
0  Industrial Conglomerates  
1  Industrial Conglomerates  
2  Industrial Conglomerates  
3  Industrial Conglomerates  
4  Industrial Conglomerates  


In [6]:
# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print("Datos preparados:")
print(f"Tamaño de X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"Tamaño de y_train: {y_train.shape}, y_test: {y_test.shape}")

Datos preparados:
Tamaño de X_train: (2205567, 5), X_test: (551392, 5)
Tamaño de y_train: (2205567, 2), y_test: (551392, 2)


In [15]:
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Crear y entrenar el modelo
modelo = RandomForestClassifier(n_estimators=20,max_depth=10,max_features= 'sqrt', random_state=42)
modelo.fit(X_train, y_train) #Tuve que reducir significativamente los parámetros por motivo de la RAM 

# Hacer predicciones en los datos de prueba
pca = PCA(n_components=5)  # Reduce a 5 componentes principales
X_test_reduced = pca.fit_transform(X_test)
y_pred = modelo.predict(X_test_reduced)

print("Dimensiones de y_test:", y_test.shape)
print("Dimensiones de y_pred:", y_pred.shape)

# Convertir y_test a un array de NumPy
y_test = np.array(y_test)
y_pred = np.array(y_pred)

for i in range(y_test.shape[1]):  
    print(f"Salida {i + 1}:")
    print("Matriz de Confusión:")
    print(confusion_matrix(y_test[:, i], y_pred[:, i]))
    print("\nReporte de Clasificación:")
    print(classification_report(y_test[:, i], y_pred[:, i]))



c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Dimensiones de y_test: (551392, 2)
Dimensiones de y_pred: (551392, 2)
Salida 1:
Matriz de Confusión:
[[    0  8399 10222     0  2351     0     5     0     2   241     0]
 [    0 18528 31912     0  4294     0     4     0    44   482     0]
 [    0 11271 29046     0  1324     0     6     0     3   267     0]
 [    0  8741 13740     0  1397     0     2     0     0   543     0]
 [    0 21417 53927     0  5121     0     2     0    27   688     0]
 [    0 18287 45461     0  5557     0     5     0    20   406     0]
 [    0 17998 58246     0  5328     0     2     0    15   295     0]
 [    0 25912 38966     0  7430     0     0     0     4   787     0]
 [    0  6831 21545     0  1518     0     1     0     0   185     0]
 [    0  6704 28141     0  1361     0     0     0     2    49     0]
 [    0  5596 30258     0   380     0     2     0     0    94     0]]

Reporte de Clasificación:


c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     21220
           1       0.12      0.34      0.18     55264
           2       0.08      0.69      0.14     41917
           3       0.00      0.00      0.00     24423
           4       0.14      0.06      0.09     81182
           5       0.00      0.00      0.00     69736
           6       0.07      0.00      0.00     81884
           7       0.00      0.00      0.00     73099
           8       0.00      0.00      0.00     30080
           9       0.01      0.00      0.00     36257
          10       0.00      0.00      0.00     36330

    accuracy                           0.10    551392
   macro avg       0.04      0.10      0.04    551392
weighted avg       0.05      0.10      0.04    551392

Salida 2:
Matriz de Confusión:
[[ 0  0  0 ...  0  0  0]
 [ 0 16  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0 12  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2495
           1       0.03      0.00      0.00     13271
           2       0.00      0.00      0.00      1284
           3       0.00      0.00      0.00      2411
           4       0.00      0.00      0.00      5026
           5       0.00      0.00      0.00      2623
           6       0.00      0.00      0.00      4452
           7       0.00      0.00      0.00     13602
           8       0.00      0.00      0.00     11389
           9       0.00      0.00      0.00      2623
          10       0.00      0.00      0.00      1924
          11       0.00      0.00      0.00      3673
          12       0.00      0.00      0.00      8425
          13       0.00      0.00      0.00      1241
          14       0.00      0.00      0.00      1548
          15       0.00      0.00      0.00      2516
          16       0.00      0.00      0.00      6683
          17       0.00    

c:\Users\Rober\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
